In [1]:
import pyspark 
import argparse 
import json 

# 4.1.1 B: The number of distinct users who have written the reviews

sc_conf = pyspark.SparkConf()
sc_conf.setAppName('task1')  
sc_conf.setMaster('local[*]') 
sc_conf.set('spark.driver.memory', '8g')
sc_conf.set('spark.executor.memory', '4g')
sc = pyspark.SparkContext(conf=sc_conf)
sc.setLogLevel("OFF")

# read in the lines and create the rdd 
lines = sc.textFile("./data/review.json")

# loading the json into a python object 
rdd = lines.map(lambda line: json.loads(line))

your 131072x1 screen size is bogus. expect trouble


23/02/01 20:17:12 WARN Utils: Your hostname, LAPTOP-0S5927HR resolves to a loopback address: 127.0.1.1; using 172.19.98.92 instead (on interface eth0)
23/02/01 20:17:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/01 20:17:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Get all unique users 

create tuple (user_id, 1) for each unique review 
Create a count from the tuples     
then sort by greatest count 

then do a select on the top m users 

In [2]:
user_review_counts = rdd.map(lambda x: (x['review_id'], x['user_id'], 1)) # each user gets 
print(type(user_review_counts))
print(user_review_counts.take(1))

<class 'pyspark.rdd.PipelinedRDD'>


[('-I5umRTkhw15RqpKMl_o1Q', '-mA3-1mN4JIEkqOtdbNXCQ', 1)]


In [4]:
# get unique reviews 
unique_reviews = sc.parallelize(user_review_counts.distinct().collect())

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 15 in stage 3.0 failed 1 times, most recent failure: Lost task 15.0 in stage 3.0 (TID 32) (172.19.98.92 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/spark-3.3.1-bin-hadoop3/python/lib/pyspark.zip/pyspark/worker.py", line 686, in main
    process()
  File "/opt/spark/spark-3.3.1-bin-hadoop3/python/lib/pyspark.zip/pyspark/worker.py", line 676, in process
    out_iter = func(split_index, iterator)
  File "/home/lucky/miniconda3/lib/python3.10/site-packages/pyspark/rdd.py", line 3472, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/home/lucky/miniconda3/lib/python3.10/site-packages/pyspark/rdd.py", line 3472, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/home/lucky/miniconda3/lib/pytho

In [ ]:
print(unique_reviews[0:3])

[('-I5umRTkhw15RqpKMl_o1Q', '-mA3-1mN4JIEkqOtdbNXCQ', 1), ('ngN7BwOpWUPA5yhmjNew7g', 'aXhMt_8ZDmfpr2QZ3zlmog', 1), ('0y6G9k9mgwORZ0oK4hTEHg', '1IhQLnq__TWCaMgDgoihJQ', 1)]
